In [37]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import KFold

from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [38]:
images = pd.read_csv("Cases\Image Segmentation\Image_Segmention.csv",)
images.head()

,Class,region.centroid.col,region.centroid.row,region.pixel.count,short.line.density.5,short.line.density.2,vedge.mean,vegde.sd,hedge.mean,hedge.sd,intensity.mean,rawred.mean,rawblue.mean,rawgreen.mean,exred.mean,exblue.mean,exgreen.mean,value.mean,saturation.mean,hue-mean
0,BRICKFACE,188,133,9,0.0,0.0,0.333333,0.266667,0.500000,0.077778,6.666666,8.333334,7.777778,3.888889,5.000000,3.333333,-8.333333,8.444445,0.538580,-0.924817
1,BRICKFACE,105,139,9,0.0,0.0,0.277778,0.107407,0.833333,0.522222,6.111111,7.555555,7.222222,3.555556,4.333334,3.333333,-7.666666,7.555555,0.532628,-0.965946
2,BRICKFACE,34,137,9,0.0,0.0,0.500000,0.166667,1.111111,0.474074,5.851852,7.777778,6.444445,3.333333,5.777778,1.777778,-7.555555,7.777778,0.573633,-0.744272
3,BRICKFACE,39,111,9,0.0,0.0,0.722222,0.374074,0.888889,0.429629,6.037037,7.000000,7.666666,3.444444,2.888889,4.888889,-7.777778,7.888889,0.562919,-1.175773
4,BRICKFACE,16,128,9,0.0,0.0,0.500000,0.077778,0.666667,0.311111,5.555555,6.888889,6.666666,3.111111,4.000000,3.333333,-7.333334,7.111111,0.561508,-0.985811


In [39]:
X = images.drop("Class",
               axis =1)
y = images["Class"]

X.head()

,region.centroid.col,region.centroid.row,region.pixel.count,short.line.density.5,short.line.density.2,vedge.mean,vegde.sd,hedge.mean,hedge.sd,intensity.mean,rawred.mean,rawblue.mean,rawgreen.mean,exred.mean,exblue.mean,exgreen.mean,value.mean,saturation.mean,hue-mean
0,188,133,9,0.0,0.0,0.333333,0.266667,0.500000,0.077778,6.666666,8.333334,7.777778,3.888889,5.000000,3.333333,-8.333333,8.444445,0.538580,-0.924817
1,105,139,9,0.0,0.0,0.277778,0.107407,0.833333,0.522222,6.111111,7.555555,7.222222,3.555556,4.333334,3.333333,-7.666666,7.555555,0.532628,-0.965946
2,34,137,9,0.0,0.0,0.500000,0.166667,1.111111,0.474074,5.851852,7.777778,6.444445,3.333333,5.777778,1.777778,-7.555555,7.777778,0.573633,-0.744272
3,39,111,9,0.0,0.0,0.722222,0.374074,0.888889,0.429629,6.037037,7.000000,7.666666,3.444444,2.888889,4.888889,-7.777778,7.888889,0.562919,-1.175773
4,16,128,9,0.0,0.0,0.500000,0.077778,0.666667,0.311111,5.555555,6.888889,6.666666,3.111111,4.000000,3.333333,-7.333334,7.111111,0.561508,-0.985811


In [40]:

le = LabelEncoder() # new from previous code
le_y = le.fit_transform(y)


print(le.classes_)

['BRICKFACE' 'CEMENT' 'FOLIAGE' 'GRASS' 'PATH' 'SKY' 'WINDOW']


In [41]:
from sklearn.pipeline import Pipeline

In [42]:
scaler = StandardScaler()
lr = LogisticRegression(solver = "saga",
                        random_state = 2022)

pipe = Pipeline([("STD", scaler), ("LR", lr)])
print(pipe.get_params())

{'memory': None, 'steps': [('STD', StandardScaler()), ('LR', LogisticRegression(random_state=2022, solver='saga'))], 'verbose': False, 'STD': StandardScaler(), 'LR': LogisticRegression(random_state=2022, solver='saga'), 'STD__copy': True, 'STD__with_mean': True, 'STD__with_std': True, 'LR__C': 1.0, 'LR__class_weight': None, 'LR__dual': False, 'LR__fit_intercept': True, 'LR__intercept_scaling': 1, 'LR__l1_ratio': None, 'LR__max_iter': 100, 'LR__multi_class': 'auto', 'LR__n_jobs': None, 'LR__penalty': 'l2', 'LR__random_state': 2022, 'LR__solver': 'saga', 'LR__tol': 0.0001, 'LR__verbose': 0, 'LR__warm_start': False}


In [43]:
params = {"LR__penalty": ["l1", "l2", "elasticnet", None],
         "LR__C": np.linspace(0.001, 4, 5),
         "LR__l1_ratio": np.linspace(0.001, 1, 5),
         "LR__multi_class":["ovr","multinomial"]}

In [45]:
kfold = StratifiedKFold(n_splits = 5,
                       shuffle = True,
                       random_state = 2022)

In [46]:
gcv = GridSearchCV(pipe,
                   param_grid = params,
                   verbose = 3,
                   scoring = "neg_log_loss",
                   cv = kfold)

gcv.fit(X, le_y)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
[CV 1/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__multi_class=ovr, LR__penalty=l1;, score=-1.946 total time=   0.0s
[CV 2/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__multi_class=ovr, LR__penalty=l1;, score=-1.946 total time=   0.0s


C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packag

[CV 3/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__multi_class=ovr, LR__penalty=l1;, score=-1.946 total time=   0.0s
[CV 4/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__multi_class=ovr, LR__penalty=l1;, score=-1.946 total time=   0.0s
[CV 5/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__multi_class=ovr, LR__penalty=l1;, score=-1.945 total time=   0.0s
[CV 1/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__multi_class=ovr, LR__penalty=l2;, score=-1.784 total time=   0.0s
[CV 2/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__multi_class=ovr, LR__penalty=l2;, score=-1.787 total time=   0.0s
[CV 3/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__multi_class=ovr, LR__penalty=l2;, score=-1.784 total time=   0.0s
[CV 4/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__multi_class=ovr, LR__penalty=l2;, score=-1.778 total time=   0.0s


C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


[CV 5/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__multi_class=ovr, LR__penalty=l2;, score=-1.764 total time=   0.0s
[CV 1/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__multi_class=ovr, LR__penalty=elasticnet;, score=-1.789 total time=   0.0s
[CV 2/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__multi_class=ovr, LR__penalty=elasticnet;, score=-1.793 total time=   0.0s
[CV 3/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__multi_class=ovr, LR__penalty=elasticnet;, score=-1.790 total time=   0.0s
[CV 4/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__multi_class=ovr, LR__penalty=elasticnet;, score=-1.785 total time=   0.0s
[CV 5/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__multi_class=ovr, LR__penalty=elasticnet;, score=-1.771 total time=   0.0s
[CV 1/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__multi_class=ovr, LR__penalty=None;, score=nan total time=   0.0s
[CV 2/5] END LR__C=0.001, LR__l1_ratio=0.001, LR__multi_class=ovr, LR__penalty=None;, score=nan total time=   0.0s
[CV 3/5] END LR__C=0.001, LR__l1_r

...

[CV 5/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__multi_class=multinomial, LR__penalty=l1;, score=-0.260 total time=   0.0s
[CV 1/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__multi_class=multinomial, LR__penalty=l2;, score=-0.368 total time=   0.0s
[CV 2/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__multi_class=multinomial, LR__penalty=l2;, score=-0.471 total time=   0.0s
[CV 3/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__multi_class=multinomial, LR__penalty=l2;, score=-0.299 total time=   0.0s
[CV 4/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__multi_class=multinomial, LR__penalty=l2;, score=-0.304 total time=   0.0s
[CV 5/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__multi_class=multinomial, LR__penalty=l2;, score=-0.311 total time=   0.0s


C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1476:

[CV 1/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__multi_class=multinomial, LR__penalty=elasticnet;, score=-0.369 total time=   0.0s
[CV 2/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__multi_class=multinomial, LR__penalty=elasticnet;, score=-0.472 total time=   0.0s
[CV 3/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__multi_class=multinomial, LR__penalty=elasticnet;, score=-0.298 total time=   0.0s
[CV 4/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__multi_class=multinomial, LR__penalty=elasticnet;, score=-0.311 total time=   0.0s
[CV 5/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__multi_class=multinomial, LR__penalty=elasticnet;, score=-0.260 total time=   0.0s
[CV 1/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__multi_class=multinomial, LR__penalty=None;, score=nan total time=   0.0s
[CV 2/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__multi_class=multinomial, LR__penalty=None;, score=nan total time=   0.0s
[CV 3/5] END LR__C=4.0, LR__l1_ratio=1.0, LR__multi_class=multinomial, LR__penalty=None;, score=nan total time=   0.0s
[CV

C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\admin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
250 fits failed out of a total of 1000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug the

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=2022, shuffle=True),
             estimator=Pipeline(steps=[('STD', StandardScaler()),
                                       ('LR',
                                        LogisticRegression(random_state=2022,
                                                           solver='saga'))]),
             param_grid={'LR__C': array([1.00000e-03, 1.00075e+00, 2.00050e+00, 3.00025e+00, 4.00000e+00]),
                         'LR__l1_ratio': array([0.001  , 0.25075, 0.5005 , 0.75025, 1.     ]),
                         'LR__multi_class': ['ovr', 'multinomial'],
                         'LR__penalty': ['l1', 'l2', 'elasticnet', None]},
             scoring='neg_log_loss', verbose=3)

In [47]:
print(gcv.best_params_)
print(gcv.best_score_)

{'LR__C': 4.0, 'LR__l1_ratio': 0.75025, 'LR__multi_class': 'multinomial', 'LR__penalty': 'elasticnet'}
-0.34094005085176576
